In [6]:
%matplotlib inline

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from atusfunclib import load_data

In [8]:
# Import all data
data_import = load_data(loc='data')

In [9]:
# Unpack individual dataframes
df, dfactcodes, dfeducodes, dfinccodes, dfagecodes, \
dfempcodes, dfindcodes, dfraccodes, dfloccodes, dfwhocodes, \
dfdemocodes = data_import

In [10]:
# Convert category columns to float
df['TEAGE_CAT'] = df['TEAGE_CAT'].astype(float)
df['TRERNWA_CAT'] = df['TRERNWA_CAT'].astype(float)

In [11]:
# Activities that we are intersted in plotting
positiveWL = ['010101', '0102', '0103', '050201',
              '050203', '1101', '1201', '1202',
              '1203', '1204', '1301', '1302', '14']

negoccWL = ['05', '0501', '0504', '1805']

neghomeWL = ['0201', '0202', '0203', '0204', '0209',
             '0301', '0302', '0303', '0304', '0305',
             '0802', '0803']

activities = positiveWL + negoccWL + neghomeWL

activities_c = ['t'+ a + '_W' for a in activities]

In [22]:
# Example work-life balance function for calculating indicator
def w_l_balance_calc(df, pos, neg):
    pos_c = ['t'+ a for a in pos]
    neg_c = ['t'+ a for a in neg]
    
    wl = df.filter(regex='^t010101$') #/ df.filter(regex='^t0803$')
    
    
    return wl

In [24]:
w_l_balance_calc(df, positiveWL, negoccWL).head()

,t010101
0,870
1,620
2,560
3,720
4,385
